In [ ]:
import jax
from jax import config
config.update("jax_enable_x64", True)

from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

from viperleed_jax.tensor_calculator import benchmark_calculator
from viperleed_jax.from_state import calculator_from_state
import importlib

jax.devices()

In [ ]:
import numpy as np
import jax
from jax.tree_util import register_pytree_node_class
from jax import numpy as jnp
from scipy import interpolate

In [ ]:
from viperleed.calc.files.displacements import readDISPLACEMENTS
from viperleed.calc import LOGGER as logger
from viperleed.calc.files.phaseshifts import readPHASESHIFTS
from viperleed.calc.files.iorfactor import beamlist_to_array

In [ ]:
%env VIPERLEED_TENSORLEED=/Users/alexander/GitHub/viperleed-tensorleed/tensorleed/

In [ ]:
input_base_path = Path('../tests/test_data/Cu_111')
large_file_path = Path('../large_files/')

origin_path = input_base_path /'dynamic_l_max'
tensor_path = large_file_path / 'Cu_111' / 'dynamic_l_max' /'Tensors' / 'Tensors_001.zip'

# origin_path = input_base_path /'fixed_l_max'
# tensor_path = large_file_path / 'Cu_111' / 'fixed_l_max' /'Tensors' / 'Tensors_001.zip'

(
    calculator,
    slab,
    rpars,
    ref_data,
    phaseshifts,
    base_scatterers,
    disp_file_from_state,
    ps_from_state,
) = calculator_from_state(
    origin_path,
    tensor_path,
    l_max=10,
    batch=100,
    recalculate_ref_t_matrices=False,
)

In [ ]:
calculator.R([0.5, 0.5, 0.5])  # Array(0.00058721, dtype=float64)

In [ ]:
ps_from_state.geo_tree

In [ ]:
calculator.ref_t_matrices.shape

In [ ]:
ps_from_state.vib_tree.ref_vib_amps

In [ ]:
calculator.R([0.5, 0.5, 0.5])

In [ ]:
ps_from_state.vib_tree

In [ ]:
ps_from_state.vib_tree.n_static_t_matrices

In [ ]:
from viperleed_jax import base_scatterers

In [ ]:
from viperleed_jax.files.displacements import regex
from viperleed_jax.files.displacements import lines
from viperleed_jax.files.displacements import file
from viperleed_jax.files.displacements import reader
from viperleed_jax.parameters import linear_transformer
from viperleed_jax.parameters import vib_parameters
from viperleed_jax.parameters import geo_parameters
from viperleed_jax.parameters import meta_parameters
from viperleed_jax.parameters import occ_parameters
from viperleed_jax.parameters import hierarchical_linear_tree
from viperleed_jax import parameter_space

In [ ]:
importlib.reload(base_scatterers)
importlib.reload(regex)
importlib.reload(lines)
importlib.reload(reader)
importlib.reload(file)
importlib.reload(linear_transformer)
importlib.reload(hierarchical_linear_tree)
importlib.reload(vib_parameters)
importlib.reload(geo_parameters)
importlib.reload(meta_parameters)
importlib.reload(occ_parameters)
importlib.reload(parameter_space)

In [ ]:
path = Path(
    "/Users/alexander/GitHub/on-the-fly-deltas/tests/test_data/displacements/Cu_111/DISPLACEMENTS_constrain_z"
)
f = file.DisplacementsFile()
f.read(path)

search_block = f.blocks[0]
offsets_block = f.offsets_block()

In [ ]:
ps = parameter_space.ParameterSpace(base_scatterers.BaseScatterers(slab), rpars)
ps.apply_displacements(offsets_block, search_block)
print(ps.info)

In [ ]:
for leaf in ps.occ_tree.leaves:
    print(leaf._bounds)

In [ ]:
ps.occ_weight_transformer.biases

In [ ]:
ps.vib_tree

In [ ]:
calculator.R([0.0, 0.5, 0.5])

In [ ]:
res = calculator.intensity([0.5, 0.5, 0.5])

In [ ]:
%matplotlib inline

In [ ]:
beam = 6
plt.plot(res[:, beam])
plt.plot(calculator.reference_intensity[:, beam])
plt.show()

In [ ]:
for node in ps.geo_tree.roots_up_to_layer(DisplacementTreeLayers.DisplacementTreeLayers.Implicit_Constraints):
    for child in node.children:
        collapsed_tansformer = child.collapse_transformer()
        print(collapsed_tansformer.weights)
        user_mask, lower, upper = child.collapse_bounds()
        print(user_mask)
        print(user_mask.sum())
        transformer = collapsed_tansformer.select_rows(user_mask)
        lower, upper = lower[user_mask], upperviperleed_jax.parameterst(transfDisplacementTreeLayersrint("---")


In [ ]:
ps.geo_tree.subtree_root.collapse_transformer().boolify()([True, True])

In [ ]:
a = np.array([[0.0]])
np.linalg.matrix_rank(a)

In [ ]:
ps.geo_tree, ps.vib_tree, ps.occ_tree

In [ ]:
from anytree.exporter import UniqueDotExporter

UniqueDotExporter(ps.geo_tree.subtree_root).to_picture("geo_subtree.pdf")

In [ ]:
from anytree.exporter import UniqueDotExporter

UniqueDotExporter(ps.vib_tree.subtree_root).to_picture("vib_subtree.pdf")

In [ ]:
list(sblock.sections.values())[0][0].direction._fractional

In [ ]:
ps.geo_tree.leaves[0].bounds

In [ ]:
print(ps.geo_tree)

In [ ]:
constrain_line = lines.ConstraintLine("geo Cu_def = linked", "geo", "Cu_def", None, "linked")

In [ ]:
vib_range_line = lines.VibDeltaLine("Cu_surf = -0.5, 0.5", "Cu_surf", None, -0.5, 0.5, None)

In [ ]:
vib_range_line.targets

In [ ]:
bs = base_scatterers.BaseScatterers(slab)
vib_subtree = vib_parameters.VibHLSubtree(bs)
geo_subtree = geo_parameters.GeoTree(bs)
occ_subtree = occ_parameters.OccTree(bs)

In [ ]:
A = np.array([[1, 0, 0],
              [1, 0, 0],
              [0, 0, 1]])

In [ ]:
#A = np.array([[1, 0, 0], [0, 0.5, -0.5]])

In [ ]:
np.linalg.matrix_rank(A)

In [ ]:
np.diag(np.linalg.qr(A).R) == 0

In [ ]:
for root in vib_subtree.roots:
    root.check_bounds_valid()

In [ ]:
vib_subtree.apply_bounds(vib_range_line)

In [ ]:
for leaf in vib_subtree.leaves:
    print(leaf.bounds)

In [ ]:
vib_subtree.apply_implicit_constraints()

In [ ]:
geo_subtree.apply_explicit_constraint(constrain_line)

In [ ]:
print(vib_subtree)

In [ ]:
# geo_subtree.create_subtree_root()
print(geo_subtree)
print(vib_subtree)
print(occ_subtree)

In [ ]:
geo_subtree.create_subtree_root()

In [ ]:
tlt = geo_subtree.subtree_root.collapse_transformer()

In [ ]:
geo_subtree.subtree_root.children[0].transformer.weights

In [ ]:
geo_subtree.subtree_root.children[1].transformer.weights

In [ ]:
tlt.weights @ np.array([False, False])

In [ ]:
np.linalg.matrix_rank(tlt.weights)

In [ ]:
tlt.biases

In [ ]:
np.linalg.qr(tlt.weights).Q

In [ ]:
np.linalg.qr(tlt.weights).R

In [ ]:
np.linalg.eig(a_test.T)

In [ ]:
a_test = geo_subtree.roots[1].collapse_transformer().weights

In [ ]:
np.linalg.matrix_rank(a_test)

In [ ]:
geo_subtree.leaves[0].parent.stacked_transformer()

In [ ]:
geo_subtree.roots

In [ ]:
DisplacementRange.DisplacementRange(-0.5, 0.5)viperleed_jax.parametersDisplacementRange

In [ ]:
l = linear_transformer.LinearTransformer(np.eye(2), np.zeros(2))

In [ ]:
l.weights

In [ ]:
print(vib_subtree)

In [ ]:

bs

In [ ]:
bs.scatterers

In [ ]:
import importlib

In [ ]:
from viperleed_jax.files.displacements import file
from viperleed_jax.files.displacements import reader
from viperleed_jax.files.displacements import regex
from viperleed_jax.files.displacements import lines
importlib.reload(regex)
importlib.reload(lines)
importlib.reload(file)
importlib.reload(reader)

In [ ]:
path = Path("/Users/alexander/GitHub/on-the-fly-deltas/tests/test_data/displacements/Cu_111/DISPLACEMENTS_constrain_z")
f = file.DisplacementsFile()
f.read(path)

In [ ]:
f

In [ ]:
b = f.blocks[0]

In [ ]:
line = b.sections[list(b.sections)[-1]][0]
line1 = b.sections[list(b.sections)[-1]][1]

In [ ]:
list(b.sections)[-1]

In [ ]:
from itertools import compress

In [ ]:
list(compress(vib_subtree.leaves, const.targets.select(bs)))


In [ ]:
const = list(b.sections.values())[-1][0]


In [ ]:
bs[const.targets.select(bs)]

In [ ]:
b.sections[DisplacementFileSections(5)]

In [ ]:
slab.ucell[:,0]

In [ ]:
slab.ab_cell @ np.array([1, 0])

In [ ]:
slab.ab_cell

In [ ]:
slab.ab_cell.volume

In [ ]:
parameter_space = ParameterSpace(slab)

## GEOMETRY
# Fix layer 0
parameter_space.geo_params.fix_layer(1, z_offset=0.)
parameter_space.geo_params.fix_layer(2, z_offset=0.)
parameter_space.geo_params.fix_layer(3, z_offset=0.)
parameter_space.geo_params.fix_layer(4, z_offset=0.)

# symmetry constrained xyz movements ± 0.15 A for layer 2
for param in [p for p in parameter_space.geo_params.terminal_params if p.bound is None]:
    param.set_bound(GeoParamBound(-0.05, + 0.05))

## VIBRATIONS
# fix *_def sites (O_def, Fe_def)
for param in [p for p in parameter_space.vib_params.terminal_params if p.site_element.site.endswith('_def')]:
    parameter_space.vib_params.fix_site_element(param.site_element, None) # None fixes to the default value


# the rest can vary ± 0.05 A
for param in [p for p in parameter_space.vib_params.terminal_params if p.site_element.site.endswith('_surf')]:
    param.set_bound(VibParamBound(-0.05, + 0.05))

## CHEMISTRY
# no free parameters
parameter_space.occ_params.remove_remaining_vacancies()

# V0R
# set ± 2 eV
parameter_space.v0r_param.set_bound(V0rParamBound(-2., +2.))

calculator.set_parameter_space(parameter_space)

In [ ]:
slab.ab_cell

In [ ]:
calculator.delta_amplitude(np.asarray((0.5, 0.5, 0.5, 0.5, 0.5)))

In [ ]:
calculator.jit_delta_amplitude(np.asarray((0.5, 0.5, 0.5, 0.5, 0.5)))

In [ ]:
calculator.R((0.5, 0.5, 0.5, 0.5, 0.5))

In [ ]:
calculator.jit_R((0.5, 0.5, 0.5, 0.5, 0.5))

In [ ]:
parameter_space

# Tree

In [ ]:
import importlib
import viperleed_jax
from viperleed_jax import parameter_space
from viperleed_jax.parameters import vib_parameters, hierarchical_linear_tree, geo_parameters, occ_parameters
from viperleed_jax.parameters import linear_transformer


In [ ]:
importlib.reload(linear_transformer)
importlib.reload(hierarchical_linear_tree)

importlib.reload(viperleed_jax)
importlib.reload(parameter_space)
importlib.reload(vib_parameters)
importlib.reload(geo_parameters)
importlib.reload(occ_parameters)

In [ ]:
from anytree import RenderTree

In [ ]:
from viperleed_jax.parameter_space import get_base_scatterers, get_site_elements

ase, se = get_base_scatterers(slab), get_site_elements(slab)

In [ ]:
vib_subtree = vib_parameters.VibHLSubtree(slab, ase, se)
vib_subtree.create_subtree_root()
print(vib_subtree)

In [ ]:
geo_subtree = geo_parameters.GeoTree(slab, ase, se)
geo_subtree.create_subtree_root()
print(geo_subtree)

In [ ]:
occ_subtree = occ_parameters.OccTree(slab, ase, se)
occ_subtree.create_subtree_root()
print(occ_subtree)

# Parameter Space and DISPLACEMENTS

In [ ]:
from importlib import reload
import viperleed_jax.files.displacements
from viperleed_jax.files.displacements.file import DisplacementsFile
reload(viperleed_jax.files.displacements)
reload(viperleed_jax.files.displacements.file)

In [ ]:
from viperleed_jax.files.displacements.file import DisplacementsFile

path = Path('/Users/alexander/GitHub/on-the-fly-deltas/tests/test_data/displacements/DISPLACEMENTS_mixed')

df = DisplacementsFile()
df.read(path)

In [ ]:
block = df.blocks[0]

In [ ]:
print(block)